# Making Neural Network Model for Addition


## Data

In [1]:
import numpy as np
import tensorflow as tf
import random
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Metric

Using TensorFlow backend.


In [2]:
def gen_data(num_examples, range_start, range_end):
    X = []
    y = []
    for ex in range(num_examples):
        num1 = int(random.random()*(range_end - range_start) + range_start)
        num2 = int(random.random()*(range_end - range_start) + range_start)

        X.append([num1,num2])        
        y.append(num1+num2)
    return np.array(X), np.array(y)

In [3]:
x_train, y_train = gen_data(10, -50, 50)
x_test, y_test = gen_data(10, 50, 200)

x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], 1,  x_test.shape[1])
[a.shape for a in [x_train, y_train, x_test, y_test]]

[(10, 1, 2), (10,), (10, 1, 2), (10,)]

## RangedAccuracy Metric

In [4]:
class RangedAccuracy(Metric):
    def __init__(self, arange=1, name="RangedAccuracy", **kwargs):
        super(RangedAccuracy, self).__init__(name=name, **kwargs)
        self.arange = tf.constant(arange, dtype=tf.float32)
        self.racc = self.add_weight(name='racc', initializer='zeros')
        self.in_range = self.add_weight(name='racc_ir', initializer='zeros')
        self.total = self.add_weight(name='racc_tot', initializer='zeros')
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_diff = tf.math.abs(y_true - y_pred)
        in_range = tf.math.less(tf.cast(y_diff, dtype=tf.float32), self.arange)
        trues = tf.reduce_sum(tf.cast(in_range, tf.float32))
        
        self.in_range = self.in_range + trues
        self.total = self.total + tf.size(in_range)
        self.racc = self.in_range / self.total
        
    
    def result(self):
        return self.racc

In [5]:
Acc = RangedAccuracy()

## Create Models

### Model with 1 layer

In [6]:
model_1layer = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=(1,2)),
    tf.keras.layers.Dense(1)
])

model_1layer.compile(
    loss="MeanSquaredError",
    metrics=['accuracy']
)

model_1layer.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 64)             192       
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              65        
Total params: 257
Trainable params: 257
Non-trainable params: 0
_________________________________________________________________


### Model with 2 Layers

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, input_shape=(1,2)),
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(1)
])
model.compile(
    loss="MeanSquaredError",
    metrics=['accuracy'] #Acc not working, in testing
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1, 128)            384       
_________________________________________________________________
dense_3 (Dense)              (None, 1, 64)             8256      
_________________________________________________________________
dense_4 (Dense)              (None, 1, 1)              65        
Total params: 8,705
Trainable params: 8,705
Non-trainable params: 0
_________________________________________________________________


### Model 3 Layer


In [8]:
model_3layer = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, input_shape=(1,2)),
  tf.keras.layers.Dense(128),
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(1)
])
model_3layer.compile(
    loss="MeanSquaredError",
    metrics=['accuracy'] #Acc not working, in testing
)

model_3layer.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1, 256)            768       
_________________________________________________________________
dense_6 (Dense)              (None, 1, 128)            32896     
_________________________________________________________________
dense_7 (Dense)              (None, 1, 64)             8256      
_________________________________________________________________
dense_8 (Dense)              (None, 1, 1)              65        
Total params: 41,985
Trainable params: 41,985
Non-trainable params: 0
_________________________________________________________________


### Model 4 Layer

In [9]:
model_4layer = tf.keras.models.Sequential([
  tf.keras.layers.Dense(512, input_shape=(1,2)),
  tf.keras.layers.Dense(256),
  tf.keras.layers.Dense(128),
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(1)
])
model_4layer.compile(
    loss="MeanSquaredError",
    metrics=['accuracy'] #Acc not working, in testing
)

model_4layer.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 1, 512)            1536      
_________________________________________________________________
dense_10 (Dense)             (None, 1, 256)            131328    
_________________________________________________________________
dense_11 (Dense)             (None, 1, 128)            32896     
_________________________________________________________________
dense_12 (Dense)             (None, 1, 64)             8256      
_________________________________________________________________
dense_13 (Dense)             (None, 1, 1)              65        
Total params: 174,081
Trainable params: 174,081
Non-trainable params: 0
_________________________________________________________________


### Model 5 Layer

In [10]:
model_5layer = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1024, input_shape=(1,2)),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(1)
])
model_5layer.compile(
    loss="MeanSquaredError",
    metrics=['accuracy'] #Acc not working, in testing
)

model_5layer.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 1, 1024)           3072      
_________________________________________________________________
dense_15 (Dense)             (None, 1, 512)            524800    
_________________________________________________________________
dense_16 (Dense)             (None, 1, 256)            131328    
_________________________________________________________________
dense_17 (Dense)             (None, 1, 128)            32896     
_________________________________________________________________
dense_18 (Dense)             (None, 1, 64)             8256      
_________________________________________________________________
dense_19 (Dense)             (None, 1, 1)              65        
Total params: 700,417
Trainable params: 700,417
Non-trainable params: 0
________________________________________________

## Train Model

In [11]:
es = EarlyStopping(monitor='val_loss', mode='min')

### Train 1 Layer Model

In [12]:
history1 = model_1layer.fit(
    x_train, y_train,
    batch_size=10000,
    epochs=100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

Epoch 1/100
1/1 [==============================] - 0s 238ms/step - loss: 3108.1526 - accuracy: 0.0000e+00 - val_loss: 136257.7969 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 49ms/step - loss: 2840.4751 - accuracy: 0.0000e+00 - val_loss: 127510.6641 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 51ms/step - loss: 2658.2520 - accuracy: 0.0000e+00 - val_loss: 120513.0391 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 51ms/step - loss: 2512.4526 - accuracy: 0.0000e+00 - val_loss: 114534.0234 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 51ms/step - loss: 2387.8572 - accuracy: 0.0000e+00 - val_loss: 109238.2734 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 49ms/step - loss: 2277.4829 - accuracy: 0.0000e+00 - val_loss: 104440.3984 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 51ms/step

Epoch 54/100
1/1 [==============================] - 0s 50ms/step - loss: 198.2194 - accuracy: 0.0000e+00 - val_loss: 8784.7432 - val_accuracy: 0.0000e+00
Epoch 55/100
1/1 [==============================] - 0s 47ms/step - loss: 182.7189 - accuracy: 0.0000e+00 - val_loss: 8079.3281 - val_accuracy: 0.0000e+00
Epoch 56/100
1/1 [==============================] - 0s 31ms/step - loss: 168.0176 - accuracy: 0.0000e+00 - val_loss: 7411.5835 - val_accuracy: 0.0000e+00
Epoch 57/100
1/1 [==============================] - 0s 47ms/step - loss: 154.1021 - accuracy: 0.0000e+00 - val_loss: 6780.8306 - val_accuracy: 0.0000e+00
Epoch 58/100
1/1 [==============================] - 0s 49ms/step - loss: 140.9581 - accuracy: 0.0000e+00 - val_loss: 6186.3462 - val_accuracy: 0.0000e+00
Epoch 59/100
1/1 [==============================] - 0s 50ms/step - loss: 128.5705 - accuracy: 0.0000e+00 - val_loss: 5627.3818 - val_accuracy: 0.0000e+00
Epoch 60/100
1/1 [==============================] - 0s 56ms/step - loss: 116

### Train 2 Layer Model

In [13]:
history2 = model.fit(
    x_train, y_train,
    batch_size=10000,
    epochs= 100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

Epoch 1/100
1/1 [==============================] - 0s 223ms/step - loss: 2290.2166 - accuracy: 0.0000e+00 - val_loss: 52168.2852 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 1068.4316 - accuracy: 0.0000e+00 - val_loss: 26987.2559 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 554.6089 - accuracy: 0.0000e+00 - val_loss: 13913.8770 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 284.0939 - accuracy: 0.0000e+00 - val_loss: 6827.0439 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 47ms/step - loss: 138.7515 - accuracy: 0.0000e+00 - val_loss: 3153.8037 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 63.4359 - accuracy: 0.0000e+00 - val_loss: 1356.2103 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 26.84

### Train 3 Layer Model

In [14]:
history3 = model_3layer.fit(
    x_train, y_train,
    batch_size=10000,
    epochs= 100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

Epoch 1/100
1/1 [==============================] - 0s 362ms/step - loss: 1946.1589 - accuracy: 0.0000e+00 - val_loss: 1931.7106 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 65ms/step - loss: 41.6376 - accuracy: 0.0000e+00 - val_loss: 282.0490 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 153ms/step - loss: 17.0334 - accuracy: 0.0000e+00 - val_loss: 162.6637 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 90ms/step - loss: 33.6494 - accuracy: 0.0000e+00 - val_loss: 116.1368 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 76ms/step - loss: 38.2581 - accuracy: 0.0000e+00 - val_loss: 90.7640 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 26ms/step - loss: 28.6873 - accuracy: 0.0000e+00 - val_loss: 109.5999 - val_accuracy: 0.0000e+00


### Train 4 Layer Model

In [15]:
history4 = model_4layer.fit(
    x_train, y_train,
    batch_size=10000,
    epochs= 100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

Epoch 1/100
1/1 [==============================] - 0s 352ms/step - loss: 2200.9519 - accuracy: 0.0000e+00 - val_loss: 123909.5469 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 60ms/step - loss: 2544.8579 - accuracy: 0.0000e+00 - val_loss: 46850.4922 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 57ms/step - loss: 995.5443 - accuracy: 0.0000e+00 - val_loss: 3316.7598 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 53ms/step - loss: 114.5033 - accuracy: 0.0000e+00 - val_loss: 372.8544 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 50ms/step - loss: 29.4392 - accuracy: 0.0000e+00 - val_loss: 45.8568 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 50ms/step - loss: 16.0580 - accuracy: 0.0000e+00 - val_loss: 39.8506 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 59ms/step - loss: 11.7724 - a

### Train 5 Layer Model

In [16]:
history5 = model_5layer.fit(
    x_train, y_train,
    batch_size=10000,
    epochs= 100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

Epoch 1/100
1/1 [==============================] - 0s 487ms/step - loss: 1471.4043 - accuracy: 0.0000e+00 - val_loss: 1761230.7500 - val_accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 56ms/step - loss: 36270.9492 - accuracy: 0.0000e+00 - val_loss: 691347.6250 - val_accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 52ms/step - loss: 14626.6748 - accuracy: 0.0000e+00 - val_loss: 26164.8359 - val_accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 50ms/step - loss: 570.4966 - accuracy: 0.0000e+00 - val_loss: 7895.1377 - val_accuracy: 0.0000e+00
Epoch 5/100
1/1 [==============================] - 0s 47ms/step - loss: 161.4714 - accuracy: 0.0000e+00 - val_loss: 1651.1914 - val_accuracy: 0.0000e+00
Epoch 6/100
1/1 [==============================] - 0s 45ms/step - loss: 33.6370 - accuracy: 0.0000e+00 - val_loss: 202.2196 - val_accuracy: 0.0000e+00
Epoch 7/100
1/1 [==============================] - 0s 46ms/step - loss: 

## Test Model

### Test 1 Layer Model

In [17]:
pred = model_1layer.predict(x_test)

In [18]:
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")

input=[115 161] expected_output=[276] prediction=[272.54428] difference=[-3.455719] %-different from actual=[0.01252072]
input=[198 122] expected_output=[320] prediction=[317.52304] difference=[-2.4769592] %-different from actual=[0.0077405]
input=[166 169] expected_output=[335] prediction=[331.5582] difference=[-3.441803] %-different from actual=[0.01027404]
input=[157  83] expected_output=[240] prediction=[238.11057] difference=[-1.8894348] %-different from actual=[0.00787264]
input=[129 197] expected_output=[326] prediction=[321.92673] difference=[-4.0732727] %-different from actual=[0.0124947]
input=[186 190] expected_output=[376] prediction=[372.2353] difference=[-3.7647095] %-different from actual=[0.01001253]
input=[111 197] expected_output=[308] prediction=[303.86996] difference=[-4.1300354] %-different from actual=[0.01340921]
input=[193 107] expected_output=[300] prediction=[297.783] difference=[-2.2170105] %-different from actual=[0.00739003]
input=[112 161] expected_output=

### Test 2 Layer Model

In [19]:
pred = model.predict(x_test)

In [20]:
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")

input=[115 161] expected_output=[276] prediction=[275.0041] difference=[-0.99591064] %-different from actual=[0.00360837]
input=[198 122] expected_output=[320] prediction=[319.3142] difference=[-0.685791] %-different from actual=[0.0021431]
input=[166 169] expected_output=[335] prediction=[334.02222] difference=[-0.9777832] %-different from actual=[0.00291876]
input=[157  83] expected_output=[240] prediction=[239.4754] difference=[-0.52459717] %-different from actual=[0.00218582]
input=[129 197] expected_output=[326] prediction=[324.82672] difference=[-1.1732788] %-different from actual=[0.00359901]
input=[186 190] expected_output=[376] prediction=[374.93292] difference=[-1.0670776] %-different from actual=[0.00283797]
input=[111 197] expected_output=[308] prediction=[306.80508] difference=[-1.1949158] %-different from actual=[0.0038796]
input=[193 107] expected_output=[300] prediction=[299.38916] difference=[-0.61083984] %-different from actual=[0.00203613]
input=[112 161] expected_ou

### Test 3 Layer Model

In [21]:
pred = model_3layer.predict(x_test)

In [22]:
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")

input=[115 161] expected_output=[276] prediction=[273.40442] difference=[-2.595581] %-different from actual=[0.00940428]
input=[198 122] expected_output=[320] prediction=[303.249] difference=[-16.751007] %-different from actual=[0.0523469]
input=[166 169] expected_output=[335] prediction=[326.26376] difference=[-8.736237] %-different from actual=[0.02607832]
input=[157  83] expected_output=[240] prediction=[225.57996] difference=[-14.420044] %-different from actual=[0.06008352]
input=[129 197] expected_output=[326] prediction=[324.4232] difference=[-1.5768127] %-different from actual=[0.00483685]
input=[186 190] expected_output=[376] prediction=[366.28482] difference=[-9.715179] %-different from actual=[0.02583824]
input=[111 197] expected_output=[308] prediction=[308.81638] difference=[0.81637573] %-different from actual=[0.00265057]
input=[193 107] expected_output=[300] prediction=[282.7137] difference=[-17.286285] %-different from actual=[0.05762095]
input=[112 161] expected_output=

### Test 4 Layer Model

In [23]:
pred = model_4layer.predict(x_test)

In [24]:
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")

input=[115 161] expected_output=[276] prediction=[270.23373] difference=[-5.766266] %-different from actual=[0.02089227]
input=[198 122] expected_output=[320] prediction=[323.91028] difference=[3.9102783] %-different from actual=[0.01221962]
input=[166 169] expected_output=[335] prediction=[332.34518] difference=[-2.6548157] %-different from actual=[0.00792482]
input=[157  83] expected_output=[240] prediction=[244.29688] difference=[4.296875] %-different from actual=[0.01790365]
input=[129 197] expected_output=[326] prediction=[318.0903] difference=[-7.9096985] %-different from actual=[0.02426288]
input=[186 190] expected_output=[376] prediction=[372.98138] difference=[-3.0186157] %-different from actual=[0.00802823]
input=[111 197] expected_output=[308] prediction=[298.74176] difference=[-9.25824] %-different from actual=[0.03005922]
input=[193 107] expected_output=[300] prediction=[304.86575] difference=[4.865753] %-different from actual=[0.01621918]
input=[112 161] expected_output=[

### Test 5 Layer Model

In [25]:
pred = model_5layer.predict(x_test)

In [26]:
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")

input=[115 161] expected_output=[276] prediction=[275.77936] difference=[-0.22064209] %-different from actual=[0.00079943]
input=[198 122] expected_output=[320] prediction=[319.834] difference=[-0.1659851] %-different from actual=[0.0005187]
input=[166 169] expected_output=[335] prediction=[334.77798] difference=[-0.22201538] %-different from actual=[0.00066273]
input=[157  83] expected_output=[240] prediction=[239.87067] difference=[-0.1293335] %-different from actual=[0.00053889]
input=[129 197] expected_output=[326] prediction=[325.74197] difference=[-0.25802612] %-different from actual=[0.00079149]
input=[186 190] expected_output=[376] prediction=[375.75787] difference=[-0.24212646] %-different from actual=[0.00064395]
input=[111 197] expected_output=[308] prediction=[307.7394] difference=[-0.2605896] %-different from actual=[0.00084607]
input=[193 107] expected_output=[300] prediction=[299.84967] difference=[-0.15032959] %-different from actual=[0.0005011]
input=[112 161] expected